In [2]:
import pandas as pd
import tskit as tsk
from IPython.display import Markdown, display
import numpy as np
import matplotlib.pyplot as plt
# from pathos.multiprocessing import ProcessingPool as Pool
from multiprocessing import Pool
from datetime import datetime
import json
import functools

%run -i "../isRecipMonophyletic.py"

filename = "cgal379.trees"
filepath = "../../data/real/"

startTime = datetime.now()

ts = tsk.load(filepath + filename)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\EK\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\EK\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\EK\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start(

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\EK\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\EK\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\EK\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start(

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



In [3]:
# Define a list of predefined colors
predefined_colors = ["red", "blue", "green", "yellow", "orange", "purple", "brown", "pink"] # Maximum 8 populations...

# Initialize node_colours dictionary
node_colours = {}

for node_index, node in enumerate(ts.tables.nodes):
	if (node.flags & tsk.NODE_IS_SAMPLE) != 0: # If node is a sample
		
		# Assign color from predefined list, cycling through colors if necessary
		color = predefined_colors[node.population % len(predefined_colors)]
		node_colours[node_index] = color



In [4]:
pop_groups = [
    [0, 1],
    [2, 3]
]

pop_by_node = pd.DataFrame({
    "pop": [ts.tables.nodes[leaf].population for leaf in ts.samples()], # Get the population of each leaf
})

In [5]:
# Create standalone pandas DataFrame for easier manipulation of data than tskit's tables...

pd_sequence = pd.DataFrame(
    {
        'span': [tree.span for tree in ts.trees()],
        'bounds': [(tree.interval.left, tree.interval.right) for tree in ts.trees()],
        'treeObj' : ts.aslist()
    },
    index=[tree.index for tree in ts.trees()]
)

### Bootstrap approach

In [6]:
n_threads = 4
n_samples = 0
multiprocessing = False

def f(i) : # Define util to be run in parallel
    np.random.seed(i + np.random.randint(0, 10000))
    bootstrap = pd_sequence.sample(n=int(len(pd_sequence) * 0.9), replace=True) # Use pandas sample method to take random trees in the sequence for monophyly test
    bootstrap['monophyletic'] = bootstrap.apply(lambda x: isRecipMonophyletic(x.treeObj, pop_by_node, pop_groups), axis=1) # Runs the test
    return bootstrap[bootstrap['monophyletic'] == True]['span'].sum() / bootstrap['span'].sum() # Gets the percentage of the sample trees that is monophyletic

if multiprocessing :
    pool = Pool(n_threads) # Create a pool of 2 workers
    percentages = np.array(pool.map(f, range(n_samples))) * 100 # Run bootstrap twice in parallel
else :
    percentages = np.array(list(map(f, range(n_samples)))) * 100

### Monophyly test for the output

In [7]:
# Runs the reciprocally monophyletic test on each tree and saves the boolean result in the monophyletic column

pd_sequence['monophyletic'] = pd_sequence.apply(lambda x: isRecipMonophyletic(ts.at_index(x.name), pop_by_node, pop_groups), axis=1)
simplepercentage = pd_sequence[pd_sequence['monophyletic'] == True]['span'].sum() / pd_sequence['span'].sum() * 100

### Export results

In [9]:
raw = {
    "tree_sequence_length": ts.last().interval.right,
    "positive": int((pd_sequence['monophyletic'] == True).sum()),
    "negative": int((pd_sequence['monophyletic'] == False).sum()),
    "total_trees": len(pd_sequence),
    "percentage": simplepercentage,
    "details": pd_sequence[['bounds', 'span', 'monophyletic']].to_dict(orient='records')
}

if n_samples > 0 :
    result = {
        "percentage": percentages.mean(),
        "stddev": percentages.std(),
        "confidence_interval": {
            "bounds": [5, 95],
            "lower": np.percentile(percentages, 5),
            "upper": np.percentile(percentages, 95)
        },
        "raw": raw
    }
else :
    result = {
        "raw": raw
    }

In [8]:
endTime = datetime.now()

In [10]:
# Define output file format

output = {
    "file": filename,
    "test": "reciprocal_monophyly",
    "description": "Reciprocal monophyly test on the tree sequence",
    "analysis_settings": {
        "bootstrap_samples": n_samples,
    },
    "perf" : {
        "multiprocessing": multiprocessing,
        "threads": n_threads,
        "start": startTime,
        "end": endTime,
        "duration": endTime - startTime
    },
    "result": result
}

In [12]:
json.dump(output, default=str, indent=4, fp=open("../../data/out/" + filename + ".recip_monophyletic.json", "w"))